In [ ]:
import general_robotics_toolbox as grt
import general_robotics_toolbox_invkin as igrt
from Arm_Lib import *
import numpy as np
import time
import math
import copy

#GLOBALS
NUM_JOINTS = 5
MIN_ANGLES = np.deg2rad(np.array([0,0,0,0,0]))
MAX_ANGLES = np.deg2rad(np.array([180,180,180,180,270]))
ZERO_CONFIG = np.deg2rad(np.array([90,90,90,90,135]))
in_2_m = 0.0254
m_2_in = 1/in_2_m

In [ ]:
def eq_config(robot, config,tolerance=872e-5):
    # robot: Robot Object
    # config: list/tuple/array of angles whose length is equal 
    #           to that of the robot's number of joints
    # tolerance: if a joint is within joint_limit and tolerance, joint snaps to the limit
    # returns: an equivalent config that is within joint limits or None if not possible
    assert(len(config) == len(robot.joint_lower_limit))
    new_angles = [None]*len(config)
    for i in range(len(config)):
        new_angles[i] = config[i]
        theta = config[i]
        if theta < robot.joint_lower_limit[i] or theta > robot.joint_upper_limit[i]:
            up = theta + np.pi*2
            dw = theta - np.pi*2
            if up > robot.joint_lower_limit[i] and up < robot.joint_upper_limit[i]:
                new_angles[i] = up
            elif dw > robot.joint_lower_limit[i] and dw < robot.joint_upper_limit[i]:
                new_angles[i] = dw
            else:
                if theta < robot.joint_lower_limit[i] and theta + tolerance >= robot.joint_lower_limit[i]:
                    new_angles[i] = robot.joint_lower_limit[i]
                elif theta > robot.joint_upper_limit[i] and theta - tolerance <= robot.joint_upper_limit[i]:
                    new_angles[i] = robot.joint_upper_limit[i]
                else:
                    return None
    return np.array(new_angles)

In [ ]:
def digest_possible_configs(robot,configs,tolerance=872e-5):
    # robot: Robot Object
    # configs: (tuple of tuples) of possible configurations
    #           for the robot
    # tolerance: see eq_config()
    # returns: all configurations that are within
    #           the joint limits
    for i in range(len(configs)):
        assert(len(robot.joint_lower_limit) == len(configs[i]))
        assert(len(robot.joint_upper_limit) == len(configs[i]))

    new_configs = [None]*len(configs)
    num_invalid = 0
    for j in range(len(configs)):
        new_configs[i] = eq_config(robot,configs[i],tolerance=tolerance)
        if new_configs[i].any() == None:
            num_invalid += 1
    if (len(configs)-num_invalid) != 0:
        rv = [None]*(len(configs)-num_invalid)
    else:
        return None
    idx = 0
    for i in range(len(configs)):
        if new_configs[i] is not None:
            rv[idx] = new_configs[i]
            idx += 1
    return np.array(rv)
        

In [ ]:
def write_arm_deg(Arm,angles,EE_angle,d_time):
        """Writes angles to arm where angles
            where angles is a numpy array of 5 integers."""
        x = angles
        Arm.Arm_serial_servo_write6(x[0],x[1],x[2],x[3],x[4],EE_angle,d_time)

In [ ]:
def write_arm_rad(Arm,angles,EE_angle,d_time):
    rd = np.deg2rad(angles)
    Arm.Arm_serial_servo_write6(rd[0],rd[1],rd[2],rd[3],rd[4],EE_angle,d_time)

In [ ]:
def getCurrentJointAngles(Arm):
    angles = np.array([0,0,0,0,0])
    for i in range(1,6):
        angles[i-1] = Arm.Arm_serial_servo_read(i)
    return angles

In [ ]:
def dofbot_robot():
    """REturns an approximate Robot instance for a DOFBOT robot"""
    x = np.array([1,0,0])
    y = np.array([0,1,0])
    z = np.array([0,0,1])
    a = np.array([0,0,0])
    
    H = np.array([z,y,y,y,-1.0*z]).T
    P = np.array([4.5*z,a,3.5*z,3.5*z,3.0*z,4.0*z]).T*in_2_m
    JTs = [0,0,0,0,0]
    return grt.Robot(H,P,JTs,MIN_ANGLES,MAX_ANGLES)

In [ ]:
def dofbot_2():
    l0 = 61 * 10**-3
    l1 = 43.5*10**-3
    l2 = 82.85 * 10**-3
    l3 = 82.85 * 10**-3
    l4 = 73.85 * 10**-3
    l5 = 0.114
    ex = np.array([1,0,0])
    ey = np.array([0,1,0])
    ez = np.array([0,0,1])
    P01 = (l0+l1) * ez
    P12 = np.zeros(3)
    P23 = l2 * ex
    P34 = -1*l3 * ez
    P45 = np.zeros(3)
    P5T = -1 *(l4+l5) * ex
    H = np.array([ez,-1*ey,-1*ey,-1*ey,-1*ex]).T
    P = np.array([P01,P12,P23,P34,P45,P5T]).T
    JTs = [0,0,0,0,0]
    lower = [0.,0.,0.,0.,0.]
    upper = np.deg2rad(np.array([180.,180.,180.,180.,270.]))
    return grt.Robot(H,P,JTs,lower,upper)

In [ ]:
def dist(P1,P2):
    # Accepts two 3X1 matrices
    # returns: distance between these two points
    diff_vec = np.absolute(P1-P2).T
    rv = 0
    for i in range(3):
        rv += diff_vec[i]*diff_vec[i]
    return rv

In [ ]:
def generate_multiple_paths(robot,desired_pose,q_current,iterations,tolerance=872e-5):
    """
        robot: Robot Object
        desired_pose: Transform Object; the desired pose of the robot -> gotten from fwdkin
        q_current: current joint angles ( in rad ) -> gotten from np.deg2rad(getCurrentJointAngles())
        iterations: positive integer -> Number of subpaths generated -> smaller means less accurate.
                    (aka 1/Lambda)
        tolerance: positive integer -> convergence tolerance ( read return below )
        returns: list of consecutive configs where the start config is the current
                    and the final config is as close to the desired_pose as possible (within tolerance)
    """
    path = np.array(np.array([q_current] + ([None]*(iterations-1))))
    initial_pose = grt.fwdkin(robot,q_current)
    poses = np.array([initial_pose] + ([None]*(iterations-1)))

    p_initial = initial_pose.p
    p_final = desired_pose.p
    r_initial = initial_pose.R
    r_final = desired_pose.R


    # Straight line towards objective
    scale_factor = 1/(iterations-1)
    p_iteration = (p_final - p_initial)*scale_factor
    r_iteration = (r_final - r_initial)*scale_factor

    for scaled_lambda in range(1,iterations):
        previous_p = poses[scaled_lambda-1].p
        previous_r = poses[scaled_lambda-1].R
        this_pose = grt.Transform(p=previous_p+p_iteration,R=previous_r+r_iteration)
        # TODO instead of just putting calculated pose -> move thena
        #       check the current pose and adjust an error metric
        poses[scaled_lambda] = this_pose
        q_invkin = igrt.iterative_invkin(robot,this_pose,path[scaled_lambda-1],tol=tolerance)
        configs = digest_possible_configs(robot,q_invkin[1])
        # TODO Select min distance config from current config

        #TODO handle config being None
        path[scaled_lambda] = configs[0]

        print("path[" +str(scaled_lambda) + "]:")
        #print(np.rad2deg(path[scaled_lambda]))

    for i in range(len(path)):
        path[i] = np.array(path[i])

    return path   

In [ ]:
def generate_path(robot, desired_pose, q_current, iterations, tolerance=872e-5):
    """
        robot: Robot Object
        desired_pose: Transform Object; the desired pose of the robot -> gotten from fwdkin
        q_current: current joint angles ( in rad ) -> gotten from np.deg2rad(getCurrentJointAngles())
        iterations: positive integer -> Number of subpaths generated -> smaller means less accurate.
                    (aka 1/Lambda)
        tolerance: positive integer -> convergence tolerance ( read return below )
        returns: list of consecutive configs where the start config is the current
                    and the final config is as close to the desired_pose as possible (within tolerance)
    """
    path = np.array(np.array([q_current] + ([None]*(iterations-1))))
    scale_factor = 1/(iterations-1)
    initial_pose = grt.fwdkin(robot,q_current)
    poses = np.array([initial_pose] + ([None]*(iterations-1)))
    
    p_initial = initial_pose.p
    p_final = desired_pose.p
    r_initial = initial_pose.R
    r_final = desired_pose.R
    p_iteration = (p_final - p_initial)*scale_factor
    r_iteration = (r_final - r_initial)*scale_factor
    for scaled_lambda in range(1,iterations):
        previous_p = poses[scaled_lambda-1].p
        previous_r = poses[scaled_lambda-1].R
        this_pose = grt.Transform(p=previous_p+p_iteration,R=previous_r+r_iteration)
        # TODO instead of just putting calculated pose -> move thena
        #       check the current pose and adjust an error metric
        poses[scaled_lambda] = this_pose
        q_invkin = igrt.iterative_invkin(robot,this_pose,path[scaled_lambda-1],tol=tolerance)
        configs = digest_possible_configs(robot,q_invkin[1])
        # TODO Select min distance config from current config

        #TODO handle config being None
        path[scaled_lambda] = configs[0]

        print("path[" +str(scaled_lambda) + "]:")
        print(np.rad2deg(path[scaled_lambda]))

    for i in range(len(path)):
        path[i] = np.array(path[i])

    return path

In [ ]:
def test_fwdkin(Arm):
    lst = []
    BOT = dofbot_2()
    print("Enter the 5 joint angles")
    for i in range(0,6):
        ele = float(input())
        lst.append(ele)
    write_arm_deg(Arm,np.array(lst),90,500)
    pose = grt.fwdkin(BOT,np.deg2rad(np.array(lst)))
    print(lst)
    print(pose.R)
    print(pose.p*in_2_m)

In [ ]:
def main(Arm):
    MEFA_BOT = dofbot_robot()
    TEST1 = [90,180,180,0,90]
    desired_pose = grt.fwdkin(MEFA_BOT, np.deg2rad(np.array(TEST1)))
    print("desired_pose:")
    print(desired_pose)
    """
    angles = np.deg2rad(getCurrentJointAngles())
    qs = igrt.iterative_invkin(MEFA_BOT,desired_pose,angles,max_steps=250)
    print("desired angles: ")
    print(TEST1)
    print(np.rad2deg(qs[1]))
    oh_god_please = digest_possible_configs(MEFA_BOT,qs[1])
    if oh_god_please is not None:
        print(np.rad2deg(oh_god_please[0]))
    """
    print("BEGIN PATH TEST")
    new_path = generate_path(MEFA_BOT,desired_pose,np.deg2rad(getCurrentJointAngles(Arm)),20)
    print("Move to path test:")
    for i in range(len(new_path)):
        write_arm_deg(Arm,np.rad2deg(new_path[i]),90,500)

   

In [ ]:
if __name__ == "__main__":
    Arm = Arm_Lib.Arm_Device()
    write_arm_deg(Arm,np.rad2deg(ZERO_CONFIG),90,500)
    print("--STARTING--")
    time.sleep(1)
    #main(Arm)
    test_fwdkin(Arm)
    print("--END--")
    del Arm
    